In [ ]:
import polars as pl
from sqlalchemy import create_engine, Column, DateTime, String
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from typing import Optional, Dict, Any
import logging
from datetime import datetime
import os
from pydantic import BaseModel, Field

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# SQLAlchemy setup
Base = declarative_base()

class DataUploadLog(Base):
    __tablename__ = 'data_upload_log'
    id = Column(String, primary_key=True)
    data_type = Column(String)
    year = Column(String)
    date_uploaded = Column(DateTime)

class DBConfig(BaseModel):
    username: str = Field(default="")
    password: str = Field(default="")
    host: str = Field(default="localhost")
    port: str = Field(default="5432")
    database: str = Field(default="sportsdata")

class DataSourceConfig(BaseModel):
    base_url: str = Field(default="https://github.com/nflverse/nflverse-data/releases/download")
    url_patterns: Dict[str, str] = Field(default_factory=dict)

class NFLVerseDataSource:
    def __init__(self, config: DataSourceConfig):
        self.config = config

    def fetch_data(self, data_type: str, year: Optional[int] = None) -> pl.DataFrame:
        if data_type not in self.config.url_patterns:
            raise ValueError(f"Unsupported data type: {data_type}")
        
        url = self.config.url_patterns[data_type]
        if "{{year}}" in url:
            if year is None:
                raise ValueError(f"Year must be provided for {data_type} data")
            url = url.format(year=year)
        
        df = pl.read_parquet(url)
        df = df.with_columns([
            pl.lit(datetime.now()).alias("date_uploaded"),
            pl.lit(data_type).alias("data_type"),
            pl.lit(str(year) if year else "N/A").alias("year")
        ])
        return df

class PostgresDataStorage:
    def __init__(self, config: DBConfig):
        self.config = config
        self.engine = create_engine(f"postgresql://{config.username}:{config.password}@{config.host}:{config.port}/{config.database}")
        Base.metadata.create_all(self.engine)
        self.Session = sessionmaker(bind=self.engine)

    def store_data(self, df: pl.DataFrame, schema: str, table_name: str, if_exists: str = "append"):
        session = self.Session()
        try:
            df.to_pandas().to_sql(table_name, self.engine, schema=schema, if_exists=if_exists, index=False)
            self._log_upload(session, table_name)
            session.commit()
            logger.info(f"Data successfully stored in {schema}.{table_name}")
        except Exception as e:
            session.rollback()
            logger.error(f"Error storing data to PostgreSQL: {str(e)}")
        finally:
            session.close()

    def _log_upload(self, session, table_name):
        log_entry = DataUploadLog(
            id=f"{table_name}_{datetime.now().strftime('%Y%m%d%H%M%S')}",
            data_type=table_name,
            year=str(datetime.now().year),
            date_uploaded=datetime.now()
        )
        session.add(log_entry)

class NFLDataManager:
    def __init__(self, data_source: NFLVerseDataSource, data_storage: PostgresDataStorage):
        self.data_source = data_source
        self.data_storage = data_storage

    def fetch_and_store_data(self, data_type: str, year: Optional[int], schema: str, table_name: str):
        try:
            df = self.data_source.fetch_data(data_type, year)
            self.data_storage.store_data(df, schema, table_name)
        except Exception as e:
            logger.error(f"Error processing {data_type} data: {str(e)}")

# Example usage:
if __name__ == "__main__":
    db_config = DBConfig(
        username=os.getenv("DB_USERNAME", ""),
        password=os.getenv("DB_PASSWORD", ""),
        host=os.getenv("DB_HOST", "localhost"),
        port=os.getenv("DB_PORT", "5432"),
        database=os.getenv("DB_NAME", "sportsdata")
    )

    data_source_config = DataSourceConfig(
        base_url="https://github.com/nflverse/nflverse-data/releases/download",
        url_patterns={
            "weekly_rosters": "{base_url}/weekly_rosters/roster_weekly_{{year}}.parquet",
            "depth_charts": "{base_url}/depth_charts/depth_charts_{{year}}.parquet",
            "pbp": "{base_url}/pbp/play_by_play_{{year}}.parquet",
            "players": "{base_url}/players/players.parquet",
            "injuries": "{base_url}/injuries/injuries_{{year}}.parquet",
        }
    )

    data_source = NFLVerseDataSource(data_source_config)
    data_storage = PostgresDataStorage(db_config)
    nfl_data_manager = NFLDataManager(data_source, data_storage)

    nfl_data_manager.fetch_and_store_data("weekly_rosters", 2023, "raw", "nflverse_weekly_rosters_2023")
    nfl_data_manager.fetch_and_store_data("depth_charts", 2023, "raw", "nflverse_depth_charts_2023")
    nfl_data_manager.fetch_and_store_data("pbp", 2023, "raw", "nflverse_pbp_2023")
    nfl_data_manager.fetch_and_store_data("players", None, "raw", "nflverse_players")